<a href="https://colab.research.google.com/github/pmugabo/Group-7-Malaria-Diagnosis-CNN-Transfer-Learning/blob/main/Jallah_VGG16_Model_Malaria_Diagnosis_CNN_Group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning for Malaria Diagnosis
This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018) and (Jason Brownlee, 2019). Acknowledge to NIH and Bangalor Hospital who make available this malaria dataset.

Malaria is an infectuous disease caused by parasites that are transmitted to people through the bites of infected female Anopheles mosquitoes.

The Malaria burden with some key figures:
<font color='red'>
* More than 219 million cases
* Over 430 000 deaths in 2017 (Mostly: children & pregnants)
* 80% in 15 countries of Africa & India
  </font>

![MalariaBurd](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaBurden.png?raw=1)

The malaria diagnosis is performed using blood test:
* Collect patient blood smear
* Microscopic visualisation of the parasit

![MalariaDiag](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/MalariaDiag.png?raw=1)
  
Main issues related to traditional diagnosis:
<font color='#ed7d31'>
* resource-constrained regions
* time needed and delays
* diagnosis accuracy and cost
</font>

The objective of this notebook is to apply modern deep learning techniques to perform medical image analysis for malaria diagnosis.

*This notebook is inspired by works of (Sivaramakrishnan Rajaraman  et al., 2018), (Adrian Rosebrock, 2018) and (Jason Brownlee, 2019)*

## Configuration

In [1]:
#Mount the local drive project_forder
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/drive/My Drive/Colab Notebooks/10xDS/Projects/malaria-diagnosis/"

Mounted at /content/drive/
ls: cannot access '/content/drive/My Drive/Colab Notebooks/10xDS/Projects/malaria-diagnosis/': No such file or directory


In [2]:
# Use GPU: Please check if the outpout is '/device:GPU:0'
import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()
#from tensorflow.python.client import device_lib
#device_lib.list_local_devices()

2.19.0


'/device:GPU:0'

## Populating namespaces

In [33]:
import os, random, json, pathlib, itertools
import numpy as np, pandas as pd, matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             confusion_matrix, roc_curve, auc)
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_class_weight

SEED=42
tf.random.set_seed(SEED); np.random.seed(SEED); random.seed(SEED)
print("TensorFlow:",tf.__version__)


TensorFlow: 2.19.0


In [4]:
# Define the useful paths for data accessibility
ai_project = '.' #"/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
cell_images_dir = os.path.join(ai_project,'cell_images')
training_path = os.path.join(ai_project,'train')
testing_path = os.path.join(ai_project,'test')

## Prepare DataSet

### *Download* DataSet

In [34]:
# Download the data in the allocated google cloud-server. If already down, turn downloadData=False
downloadData = True
if downloadData == True:
  indrive = False
  if indrive == True:
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip -P "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
    !unzip "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/cell_images.zip" -d "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis/"
    !ls "/content/drive/My Drive/Colab Notebooks/ai-labs/malaria-diagnosis"
  else: #incloud google server
    !rm -rf cell_images.*
    !wget https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
    !unzip cell_images.zip >/dev/null 2>&1
    !ls

--2025-10-05 16:36:42--  https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
Resolving data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)... 3.165.102.96, 3.165.102.59, 3.165.102.109, ...
Connecting to data.lhncbc.nlm.nih.gov (data.lhncbc.nlm.nih.gov)|3.165.102.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353452851 (337M) [application/zip]
Saving to: ‘cell_images.zip’

cell_images.zip     100%[===================>] 337.08M   393MB/s    in 0.9s    

2025-10-05 16:36:43 (393 MB/s) - ‘cell_images.zip’ saved [353452851/353452851]

cell_images	 drive		sample_data
cell_images.zip  malaria_split	vgg16_experiments


## Baseline CNN Model
Define a basic ConvNet defined with ConvLayer: Conv2D => MaxPooling2D followed by Flatten => Dense => Dense(output)

![ConvNet](https://github.com/habiboulaye/ai-labs/blob/master/malaria-diagnosis/doc-images/ConvNet.png?raw=1)


# Transfer Learning Model: VGG16

**Objective:**

1.   Implement and fine-tune VGG16 for malaria cell classification.
2.   Conduct seven experiments systematically varying augmentation, fine-tuning depth, optimizer, and regularization.
3.   Evaluate models rigorously using accuracy, precision, recall, F1, and
2.   visualize learning curves, confusion matrices, and ROC/AUC.





VGG16’s simple, uniform 3×3 conv stacks form a strong baseline and a common medical-imaging transfer model. Despite being parameter-heavy, it transfers well when paired with global pooling, dropout, and careful learning-rate schedules.

In [1]:
import os, json, random, itertools, pathlib, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from sklearn.utils.class_weight import compute_class_weight

# Reproducibility
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

RAW_ROOT   = "/content/cell_images"
SPLIT_ROOT = "/content/cell_images_split"
OUTPUT_DIR = "/content/vgg16_experiments"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("TensorFlow:", tf.__version__)


TensorFlow: 2.19.0


Train a VGG16 model, run at least seven experiments with different configurations, document the choices made, evaluate the model rigorously using accuracy, precision, recall, and F1-score presented in tables, and include visual evidence of evaluation such as learning curves, confusion matrices, and ROC/AUC curves.

## Prepare data

Split the dataset into training, validation, and test sets.


In [3]:
# Remove the split directory if it exists
if os.path.exists(SPLIT_ROOT):
    shutil.rmtree(SPLIT_ROOT)

# Create the directory structure for the split dataset
for split_name in ['train', 'val', 'test']:
    for class_name in ['Parasitized', 'Uninfected']:
        os.makedirs(os.path.join(SPLIT_ROOT, split_name, class_name), exist_ok=True)

# Get the list of image files and shuffle them
parasitized_files = list(pathlib.Path(RAW_ROOT, 'Parasitized').glob('*.png'))
uninfected_files = list(pathlib.Path(RAW_ROOT, 'Uninfected').glob('*.png'))

random.shuffle(parasitized_files)
random.shuffle(uninfected_files)

# Determine the number of files for each split
total_parasitized = len(parasitized_files)
total_uninfected = len(uninfected_files)

train_split = 0.8
val_split = 0.1
test_split = 0.1

train_parasitized_count = int(total_parasitized * train_split)
val_parasitized_count = int(total_parasitized * val_split)
test_parasitized_count = total_parasitized - train_parasitized_count - val_parasitized_count

train_uninfected_count = int(total_uninfected * train_split)
val_uninfected_count = int(total_uninfected * val_split)
test_uninfected_count = total_uninfected - train_uninfected_count - val_uninfected_count

# Copy the files to the respective directories
def copy_files(file_list, destination_dir):
    for file_path in file_list:
        shutil.copy(file_path, destination_dir)

# Copy Parasitized files
copy_files(parasitized_files[:train_parasitized_count], os.path.join(SPLIT_ROOT, 'train', 'Parasitized'))
copy_files(parasitized_files[train_parasitized_count:train_parasitized_count + val_parasitized_count], os.path.join(SPLIT_ROOT, 'val', 'Parasitized'))
copy_files(parasitized_files[train_parasitized_count + val_parasitized_count:], os.path.join(SPLIT_ROOT, 'test', 'Parasitized'))

# Copy Uninfected files
copy_files(uninfected_files[:train_uninfected_count], os.path.join(SPLIT_ROOT, 'train', 'Uninfected'))
copy_files(uninfected_files[train_uninfected_count:train_uninfected_count + val_uninfected_count], os.path.join(SPLIT_ROOT, 'val', 'Uninfected'))
copy_files(uninfected_files[train_uninfected_count + val_uninfected_count:], os.path.join(SPLIT_ROOT, 'test', 'Uninfected'))

print("Dataset split and copied successfully.")

Dataset split and copied successfully.


## Define model building function

Create a function to build and compile the VGG16 model with customizable layers, optimizer, and regularization.


In [4]:
def build_vgg16_model(trainable_layers=-1, optimizer='adam', learning_rate=0.001, l2_strength=0.0, dropout_rate=0.0):
    """
    Builds and compiles a VGG16 model with customizable layers, optimizer, and regularization.

    Args:
        trainable_layers (int): Number of VGG16 layers to unfreeze for training.
                                 -1: All layers are trainable.
                                  0: None of the VGG16 layers are trainable.
                                 >0: The last 'trainable_layers' are unfrozen.
        optimizer (str): The name of the optimizer ('adam' or 'sgd').
        learning_rate (float): The learning rate for the optimizer.
        l2_strength (float): The L2 regularization strength for dense layers.
        dropout_rate (float): The dropout rate for the dropout layer.

    Returns:
        keras.Model: The compiled Keras model.
    """
    # Load the VGG16 base model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Set the number of trainable layers in the base model
    if trainable_layers == 0:
        base_model.trainable = False
    elif trainable_layers > 0:
        for layer in base_model.layers[:-trainable_layers]:
            layer.trainable = False
        for layer in base_model.layers[-trainable_layers:]:
            layer.trainable = True
    else: # trainable_layers == -1
         base_model.trainable = True


    # Add custom layers on top of the base model
    model = keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(l2_strength)),
        layers.Dropout(dropout_rate),
        layers.Dense(1, activation='sigmoid')
    ])

    # Define the optimizer
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        raise ValueError(f"Optimizer '{optimizer}' not supported. Choose 'adam' or 'sgd'.")


    # Compile the model
    model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model


## Define experiment configurations

Define at least seven different configurations for the experiments, varying parameters such as data augmentation, fine-tuning depth, optimizer, and regularization. I will define a list of dictionaries, each representing an experiment configuration with varying parameters like data augmentation, fine-tuning depth, optimizer, learning rate, L2 regularization, and dropout rate. This list will be used to run multiple experiments.




In [5]:
experiment_configurations = [
    {
        "experiment_name": "exp_1_no_aug_freeze_all_adam",
        "data_augmentation": False,
        "trainable_layers": 0, # Freeze all VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.001,
        "l2_strength": 0.0,
        "dropout_rate": 0.0
    },
    {
        "experiment_name": "exp_2_aug_freeze_all_adam",
        "data_augmentation": True,
        "trainable_layers": 0, # Freeze all VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.001,
        "l2_strength": 0.0,
        "dropout_rate": 0.0
    },
    {
        "experiment_name": "exp_3_aug_unfreeze_last_adam",
        "data_augmentation": True,
        "trainable_layers": 4, # Unfreeze the last few VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.0001, # Lower learning rate for fine-tuning
        "l2_strength": 0.0,
        "dropout_rate": 0.0
    },
     {
        "experiment_name": "exp_4_aug_unfreeze_all_adam",
        "data_augmentation": True,
        "trainable_layers": -1, # Unfreeze all VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.00001, # Very low learning rate for full fine-tuning
        "l2_strength": 0.0,
        "dropout_rate": 0.0
    },
    {
        "experiment_name": "exp_5_aug_unfreeze_last_sgd",
        "data_augmentation": True,
        "trainable_layers": 4, # Unfreeze the last few VGG16 layers
        "optimizer": "sgd",
        "learning_rate": 0.001,
        "l2_strength": 0.0,
        "dropout_rate": 0.0
    },
    {
        "experiment_name": "exp_6_aug_unfreeze_last_adam_l2",
        "data_augmentation": True,
        "trainable_layers": 4, # Unfreeze the last few VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.0001,
        "l2_strength": 0.001, # Add L2 regularization
        "dropout_rate": 0.0
    },
    {
        "experiment_name": "exp_7_aug_unfreeze_last_adam_dropout",
        "data_augmentation": True,
        "trainable_layers": 4, # Unfreeze the last few VGG16 layers
        "optimizer": "adam",
        "learning_rate": 0.0001,
        "l2_strength": 0.0,
        "dropout_rate": 0.5 # Add dropout
    }
]

print(f"Defined {len(experiment_configurations)} experiment configurations.")
# Display the configurations
for config in experiment_configurations:
    print(config)

Defined 7 experiment configurations.
{'experiment_name': 'exp_1_no_aug_freeze_all_adam', 'data_augmentation': False, 'trainable_layers': 0, 'optimizer': 'adam', 'learning_rate': 0.001, 'l2_strength': 0.0, 'dropout_rate': 0.0}
{'experiment_name': 'exp_2_aug_freeze_all_adam', 'data_augmentation': True, 'trainable_layers': 0, 'optimizer': 'adam', 'learning_rate': 0.001, 'l2_strength': 0.0, 'dropout_rate': 0.0}
{'experiment_name': 'exp_3_aug_unfreeze_last_adam', 'data_augmentation': True, 'trainable_layers': 4, 'optimizer': 'adam', 'learning_rate': 0.0001, 'l2_strength': 0.0, 'dropout_rate': 0.0}
{'experiment_name': 'exp_4_aug_unfreeze_all_adam', 'data_augmentation': True, 'trainable_layers': -1, 'optimizer': 'adam', 'learning_rate': 1e-05, 'l2_strength': 0.0, 'dropout_rate': 0.0}
{'experiment_name': 'exp_5_aug_unfreeze_last_sgd', 'data_augmentation': True, 'trainable_layers': 4, 'optimizer': 'sgd', 'learning_rate': 0.001, 'l2_strength': 0.0, 'dropout_rate': 0.0}
{'experiment_name': 'exp_6

## Run experiments

Iterate through the experiment configurations, train the VGG16 model for each configuration, and save the training history and model weights.


In [ ]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

for config in experiment_configurations:
    exp_name = config['experiment_name']
    data_augmentation = config['data_augmentation']
    trainable_layers = config['trainable_layers']
    optimizer = config['optimizer']
    learning_rate = config['learning_rate']
    l2_strength = config['l2_strength']
    dropout_rate = config['dropout_rate']

    print(f"\nStarting experiment: {exp_name}")

    # Create experiment output directory
    exp_output_dir = os.path.join(OUTPUT_DIR, exp_name)
    os.makedirs(exp_output_dir, exist_ok=True)

    # Save configuration
    with open(os.path.join(exp_output_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)

    # Define data generators
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    ) if data_augmentation else tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1./255
    )

    val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1./255
    )

    test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rescale=1./255
    )


    train_generator = train_datagen.flow_from_directory(
        os.path.join(SPLIT_ROOT, 'train'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    val_generator = val_datagen.flow_from_directory(
        os.path.join(SPLIT_ROOT, 'val'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    test_generator = test_datagen.flow_from_directory(
        os.path.join(SPLIT_ROOT, 'test'),
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='binary',
        seed=SEED
    )

    # Compute class weights
    classes = np.unique(train_generator.classes)
    class_weights = compute_class_weight(
        class_weight='balanced',
        classes=classes,
        y=train_generator.classes
    )
    class_weight_dict = dict(zip(classes, class_weights))
    print("Class weights:", class_weight_dict)

    # Build the model
    model = build_vgg16_model(
        trainable_layers=trainable_layers,
        optimizer=optimizer,
        learning_rate=learning_rate,
        l2_strength=l2_strength,
        dropout_rate=dropout_rate
    )

    # Define callbacks
    checkpoint_filepath = os.path.join(exp_output_dir, 'best_model.weights.h5')
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True
    )

    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    )

    callbacks = [model_checkpoint_callback, early_stopping_callback]

    # Train the model
    history = model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weight_dict,
        callbacks=callbacks
    )

    # Save training history
    history_dict = history.history
    with open(os.path.join(exp_output_dir, 'history.json'), 'w') as f:
        json.dump(history_dict, f)

    # Save the final model weights
    model.save_weights(os.path.join(exp_output_dir, 'final_model.weights.h5'))

    print(f"Experiment {exp_name} finished.")


Starting experiment: exp_1_no_aug_freeze_all_adam
Found 22046 images belonging to 2 classes.
Found 2754 images belonging to 2 classes.
Found 2758 images belonging to 2 classes.
Class weights: {np.int32(0): np.float64(1.0), np.int32(1): np.float64(1.0)}
Epoch 1/10


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


689/689 ━━━━━━━━━━━━━━━━━━━━ 134s 190ms/step - accuracy: 0.7580 - loss: 0.4939 - val_accuracy: 0.9049 - val_loss: 0.2593
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9015 - loss: 0.2588 - val_accuracy: 0.9096 - val_loss: 0.2271
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9078 - loss: 0.2396 - val_accuracy: 0.9168 - val_loss: 0.2087
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9199 - loss: 0.2099 - val_accuracy: 0.8744 - val_loss: 0.3004
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9196 - loss: 0.2083 - val_accuracy: 0.9194 - val_loss: 0.2024
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9222 - loss: 0.1989 - val_accuracy: 0.9158 - val_loss: 0.2127
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9196 - loss: 0.2033 - val_accuracy: 0.9256 - val_loss: 0.1880
Epoch 8/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 126s 183ms/step - accuracy: 0.9290 - loss: 0.19

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 323s 464ms/step - accuracy: 0.7530 - loss: 0.4988 - val_accuracy: 0.8936 - val_loss: 0.2943
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 317s 460ms/step - accuracy: 0.8908 - loss: 0.2806 - val_accuracy: 0.9016 - val_loss: 0.2593
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 320s 464ms/step - accuracy: 0.9088 - loss: 0.2430 - val_accuracy: 0.9143 - val_loss: 0.2309
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 322s 467ms/step - accuracy: 0.9068 - loss: 0.2348 - val_accuracy: 0.9161 - val_loss: 0.2238
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 320s 464ms/step - accuracy: 0.9086 - loss: 0.2358 - val_accuracy: 0.9147 - val_loss: 0.2230
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 320s 464ms/step - accuracy: 0.9148 - loss: 0.2184 - val_accuracy: 0.9147 - val_loss: 0.2170
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 329s 476ms/step - accuracy: 0.9184 - loss: 0.2146 - val_accuracy: 0.9081 - val_loss: 0.2346
Epoch 8/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 321s 465ms/step - accuracy: 0.9205 -

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 334s 476ms/step - accuracy: 0.8882 - loss: 0.2679 - val_accuracy: 0.9535 - val_loss: 0.1389
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 324s 471ms/step - accuracy: 0.9544 - loss: 0.1356 - val_accuracy: 0.9539 - val_loss: 0.1433
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 331s 480ms/step - accuracy: 0.9551 - loss: 0.1285 - val_accuracy: 0.9579 - val_loss: 0.1263
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 323s 468ms/step - accuracy: 0.9588 - loss: 0.1239 - val_accuracy: 0.9604 - val_loss: 0.1208
Epoch 5/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 325s 471ms/step - accuracy: 0.9549 - loss: 0.1237 - val_accuracy: 0.9608 - val_loss: 0.1212
Epoch 6/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 319s 463ms/step - accuracy: 0.9597 - loss: 0.1136 - val_accuracy: 0.9582 - val_loss: 0.1219
Epoch 7/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 322s 468ms/step - accuracy: 0.9613 - loss: 0.1111 - val_accuracy: 0.9601 - val_loss: 0.1108
Epoch 8/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 324s 471ms/step - accuracy: 0.9648 -

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 489s 647ms/step - accuracy: 0.8679 - loss: 0.2894 - val_accuracy: 0.9601 - val_loss: 0.1207
Epoch 2/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 416s 603ms/step - accuracy: 0.9607 - loss: 0.1167 - val_accuracy: 0.9659 - val_loss: 0.1030
Epoch 3/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 419s 608ms/step - accuracy: 0.9640 - loss: 0.1054 - val_accuracy: 0.9637 - val_loss: 0.1021
Epoch 4/10
689/689 ━━━━━━━━━━━━━━━━━━━━ 420s 609ms/step - accuracy: 0.9645 - loss: 0.1055 - val_accuracy: 0.9662 - val_loss: 0.0956
Epoch 5/10
262/689 ━━━━━━━━━━━━━━━━━━━━ 4:10 587ms/step - accuracy: 0.9653 - loss: 0.0976